# Intro

Here we will show you how to use some of the basic functionalities of Bokeh, a python visualization library.
Bokeh, Plotly, so forth are libraries that can be used to create rich, interative and ready to be served to web-pages.

Bokeh is open-source, which means that you can see how it is built, and you can also help built it!

## Lines !
The forefather of plots.
Lets just plot a line.

First we need to import the necessary packages from Bokeh. 

Figure is one of the basic foundations of Bokeh.

In [2]:
from bokeh.plotting import figure,show,output_file
from bokeh.io import output_notebook


Run this so that the plots are shown here

In [3]:
output_notebook()

Loading BokehJS ...

First we need some data

In [19]:
x = [i for i in range(0,20)]
y = [20-i for i in range(0,20)]

Then we create the figure

In [20]:
fig = figure(title = "My first plot!",width=400,height=400)

Now, we add a line and show the plot

In [21]:
fig.line(x,y)
show(fig)

There you go, a normal plot, nothing new, but do you notice those icons on the right side? Hover over them and try to use them.

These icons stand for Pan, Box Zoom, Wheel Zoom, Save and Reset.Which are tools that are built-in to your plots easily. Keep learning, we will use more of these!

Lets add another line!

In [22]:
fig.line(x,x)
show(fig)


When we have several lines on a plot we can differentiate them with legends and colors, lets do that.

This time we just want a simple plot, no tools included!

In [25]:
fig = figure(title = "My first plot!",width=400,height=400,tools="")
fig.line(x,y,color="red",legend_label="red")
fig.line(x,x,color="blue",legend_label="blue")
show(fig)

These lines share the same x, with bokeh we can simplify the way we do this.
And this time we want to be able to zoom, but only on the x axis.

In [35]:
fig = figure(title = "My first plot!",width=400,height=400,tools="xwheel_zoom,reset,pan")

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y1=[1, 2, 4, 3, 4],
    y2=[1, 4, 2, 2, 3],
))

colors = ["orange","red"]
fig.vline_stack(["y1","y2"],x='x',source=source, color=colors,legend_label = colors) 
show(fig)

### Question
Uh, a ColumnDataSource appeared out of nowhere?  What is that for?

## Bands
What is your favourite  Band?


We have to import a lil' something here first..

In [45]:
from bokeh.models import Range1d, Band

Here we setup our data, setup our plot and add our band.

In [48]:
x = [i for i in range(10)]
fig = figure(title = "Not Music Bands",width=400,height=400,tools="xwheel_zoom,reset,pan,lasso_select",x_range=Range1d(4,8,bounds=(-1,11)),y_range=(0,10))

source = ColumnDataSource(data=dict(
    x=x,
    lower=[i-1 for i in x], 
    upper=[i+1 for i in x]  
))
band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
            fill_alpha=0.2, line_width=1, line_color='black',fill_color="orange")

fig.add_layout(band) #we need to add this one explicitly..
show(fig)

Pss pss, try to pan around the x axis..

Bands are really usefull in data visualization to describe variations of the data. 

For example, one might use a Line to plot the mean and then place a Land over it to display the variation of the data!